In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [73]:
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, SVD,SVDpp, NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_split_surprise
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import ndcg_score
import scipy.stats as ss

In [74]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

from src.utilities import *
from src.reccomendation_util import *

In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Model and Raw Data

In [78]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
print(raw_interactions.shape)

(1132367, 5)


In [71]:
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata.shape

(231637, 13)

In [38]:
### Load Models
model1 = load_pickle("../../models/reccomender_model1_svd.pkl")
model2 = load_pickle("../../models/reccomender_model2_nmf.pkl")

In [55]:
model1.predict(uid=1533,iid=44459)

Prediction(uid=1533, iid=44459, r_ui=None, est=4.979205256134577, details={'was_impossible': False})

In [54]:
model1.test([[1533,57549,None],\
            [1533,44459,None],\
            [1533,31048,None],\
            [1533,10554,None]])

[Prediction(uid=1533, iid=57549, r_ui=None, est=4.984623436267778, details={'was_impossible': False}),
 Prediction(uid=1533, iid=44459, r_ui=None, est=4.979205256134577, details={'was_impossible': False}),
 Prediction(uid=1533, iid=31048, r_ui=None, est=4.909425973113856, details={'was_impossible': False}),
 Prediction(uid=1533, iid=10554, r_ui=None, est=4.904533254551073, details={'was_impossible': False})]

In [69]:
model2.test([[1533,57549,None],\
            [1533,44459,None],\
            [1533,31048,None],\
            [1533,10554,None]])

[Prediction(uid=1533, iid=57549, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=44459, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=31048, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=10554, r_ui=None, est=5, details={'was_impossible': False})]

### Reccomendation Testting

In [80]:
gen_recipe_list(raw_interactions,1533)

array([ 40893,  44394,  85009, ..., 257796,  72730, 386618])

In [82]:
get_reccomendation_ids(model1,1533,gen_recipe_list(raw_interactions,1533),20)

[123665,
 27509,
 379639,
 49461,
 265683,
 235347,
 119804,
 515167,
 138647,
 319562,
 97606,
 92649,
 524863,
 192308,
 152898,
 61195,
 72119,
 24044,
 16603,
 134316]

In [83]:
rec_ids = get_reccomendation_ids(model1,1533,gen_recipe_list(raw_interactions,1533),20)

In [86]:
recipe_meta_map(recipe_metadata,rec_ids)

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation/../../src/reccomendation_util.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return df_subset


,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,recipe_id
165774,165774,preserved lemons,16603,7205,21752,2002-01-02,"['weeknight', 'time-to-make', 'course', 'main-...","[16.8, 1.0, 2.0, 65.0, 0.0, 0.0, 0.0]",8,"['blanch 6 lemons in boiling water 5 minutes',...",this is an adaptation of paula wolfert's quick...,"['lemons', 'coarse salt', 'olive oil']",3,16603
140186,140186,mrs schreiner s split pea soup,24044,140,8688,2002-03-31,"['weeknight', 'time-to-make', 'course', 'main-...","[306.3, 16.0, 22.0, 21.0, 33.0, 17.0, 12.0]",9,['in a large heavy kettle or stock pot cook ba...,my family loves split-pea soup and this is the...,"['bacon', 'onion', 'carrots', 'dried split pea...",8,24044
125585,125585,lost bread french toast,27509,25,36998,2002-05-07,"['30-minutes-or-less', 'time-to-make', 'course...","[234.3, 7.0, 161.0, 2.0, 10.0, 10.0, 14.0]",10,['mix both dry ingredients until thoroughly co...,"when i was a little girl, my mama always made ...","['sugar', 'cinnamon', 'eggs', 'milk', 'vanilla...",7,27509
15483,15483,baked vegetarian chimichangas warm or cold,49461,25,39547,2002-12-22,"['30-minutes-or-less', 'time-to-make', 'course...","[250.1, 15.0, 15.0, 24.0, 23.0, 26.0, 9.0]",9,"['saut the chopped onion , garlic , and mushro...","these are a deliciously healthy, filling meal,...","['onion', 'garlic cloves', 'mushroom', 'olive ...",17,49461
139891,139891,moxie s crystallized ginger cookies,61195,85,55380,2003-05-05,"['weeknight', 'time-to-make', 'course', 'main-...","[90.4, 6.0, 27.0, 4.0, 1.0, 11.0, 4.0]",9,"['preheat oven to 375', 'cream butter& sesame ...",this was originally posted by my cyber-friend ...,"['unsalted butter', 'sesame oil', 'dark molass...",12,61195
185902,185902,shrimp parmesan,72119,55,58104,2003-10-01,"['60-minutes-or-less', 'time-to-make', 'course...","[860.7, 42.0, 40.0, 92.0, 110.0, 68.0, 31.0]",17,['open the shrimp up like a book and remove th...,my husband favorite. a one dish meal. you can ...,"['large shrimp', 'flour', 'old bay seasoning',...",12,72119
26220,26220,blueberry pancakes milk free egg free,92649,15,40135,2004-06-04,"['lactose', '15-minutes-or-less', 'time-to-mak...","[162.3, 8.0, 30.0, 12.0, 6.0, 6.0, 8.0]",15,"['sift flour , sugar , baking powder , and sal...",a seriously delicious recipe. we pick our wild...,"['white flour', 'white sugar', 'baking powder'...",8,92649
59201,59201,country sausage gravy,97606,35,134663,2004-08-12,"['60-minutes-or-less', 'time-to-make', 'course...","[378.2, 46.0, 0.0, 32.0, 32.0, 65.0, 3.0]",8,['brown the pork sausage in a large skillet ov...,"growing up, we ate this gravy over slices of w...","['pork sausage', 'butter', 'all-purpose flour'...",6,97606
117962,117962,kittencal s banana cinnamon snack cake or muff...,119804,30,89831,2005-04-30,"['30-minutes-or-less', 'time-to-make', 'course...","[183.1, 5.0, 59.0, 15.0, 9.0, 3.0, 10.0]",11,"['set oven to 350 degrees f', 'grease a 8 or 9...",a wonderful lower-fat cake/muffin recipe that'...,"['all-purpose flour', 'sugar', 'baking powder'...",11,119804
32135,32135,buttermilk buckwheat pancakes gluten free,123665,25,157102,2005-05-26,"['30-minutes-or-less', 'time-to-make', 'course...","[195.9, 8.0, 41.0, 27.0, 14.0, 13.0, 10.0]",6,"['combine dry ingredients in large bowl', 'whi...",i’m posting this mainly for those of you on za...,"['buckwheat flour', 'brown sugar', 'baking pow...",11,123665
